In [32]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from scrapper import fetch_website_links, fetch_website_contents
from IPython.display import Markdown, display ,update_display
import json

In [4]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
Model = "gpt-4o-mini"


In [6]:
links=fetch_website_links("https://www.apple.com")
links

['/',
 '/us/shop/goto/store',
 '/mac/',
 '/ipad/',
 '/iphone/',
 '/watch/',
 '/apple-vision-pro/',
 '/airpods/',
 '/tv-home/',
 '/entertainment/',
 '/us/shop/goto/buy_accessories',
 'https://support.apple.com/?cid=gn-ols-home-hp-tab',
 '/us/search',
 '/us/shop/goto/bag',
 '/us/shop/goto/gifts/shopping_event',
 '/us/shop/goto/gifts/shopping_event',
 '/iphone-17-pro/',
 '/iphone-17-pro/',
 '/us/shop/goto/buy_iphone/iphone_17_pro',
 '/iphone-air/',
 '/iphone-air/',
 '/us/shop/goto/buy_iphone/iphone_air',
 '/us/shop/goto/store',
 '/us/shop/goto/store',
 '/us/shop/goto/store',
 '/us/shop/goto/store',
 '/airpods-pro/',
 '/airpods-pro/',
 '/us/shop/goto/buy_airpods/airpods_pro_3',
 '/ipad-air/',
 '/ipad-air/',
 '/us/shop/goto/buy_ipad/ipad_air',
 '/apple-watch-series-11/',
 '/apple-watch-series-11/',
 '/us/shop/goto/buy_watch/apple_watch_series_11',
 '/macbook-pro/',
 '/macbook-pro/',
 '/us/shop/goto/buy_mac/macbook_pro/14_inch',
 '/us/shop/goto/buy_iphone/carrier_offers',
 '/us/shop/goto/buy

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [11]:
def get_link_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links=fetch_website_links("https://www.apple.com")
    user_prompt += "\n".join(links)
    return user_prompt



In [12]:
get_link_user_prompt("https://apple.com")

'\nHere is the list of links on the website https://apple.com -\nPlease decide which of these are relevant web links for a brochure about the company, \nrespond with the full https URL in JSON format.\nDo not include Terms of Service, Privacy, email links.\n\nLinks (some might be relative links):\n\n/\n/us/shop/goto/store\n/mac/\n/ipad/\n/iphone/\n/watch/\n/apple-vision-pro/\n/airpods/\n/tv-home/\n/entertainment/\n/us/shop/goto/buy_accessories\nhttps://support.apple.com/?cid=gn-ols-home-hp-tab\n/us/search\n/us/shop/goto/bag\n/us/shop/goto/gifts/shopping_event\n/us/shop/goto/gifts/shopping_event\n/iphone-17-pro/\n/iphone-17-pro/\n/us/shop/goto/buy_iphone/iphone_17_pro\n/iphone-air/\n/iphone-air/\n/us/shop/goto/buy_iphone/iphone_air\n/us/shop/goto/store\n/us/shop/goto/store\n/us/shop/goto/store\n/us/shop/goto/store\n/airpods-pro/\n/airpods-pro/\n/us/shop/goto/buy_airpods/airpods_pro_3\n/ipad-air/\n/ipad-air/\n/us/shop/goto/buy_ipad/ipad_air\n/apple-watch-series-11/\n/apple-watch-series-1

In [14]:
def generate_usefull_links(url):
    user_prompt = get_link_user_prompt(url)
    response = client.chat.completions.create(
        model=Model,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
    return json.loads(response.choices[0].message.content)






In [15]:
generate_usefull_links("https://apple.com")

{'links': [{'type': 'about page', 'url': 'https://apple.com/about'},
  {'type': 'careers page', 'url': 'https://apple.com/careers/us/'},
  {'type': 'company page', 'url': 'https://apple.com/newsroom/'},
  {'type': 'leadership page', 'url': 'https://apple.com/leadership/'},
  {'type': 'investor relations', 'url': 'https://investor.apple.com/'},
  {'type': 'business page', 'url': 'https://apple.com/business/'},
  {'type': 'education page', 'url': 'https://apple.com/education/'},
  {'type': 'healthcare page', 'url': 'https://apple.com/healthcare/'},
  {'type': 'government page', 'url': 'https://apple.com/government/'}]}

``` Time for brochure```

In [18]:
def generate_page_data_with_links(url):
    links = generate_usefull_links(url)
    page_data = fetch_website_contents(url)
    result= f"## landing page\n\n{page_data}\n\n## links\n"
    for link in links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link['url'])
    return result




In [20]:
print(generate_page_data_with_links("https://apple.com"))

## landing page

Apple

Apple
Apple
Store
Mac
iPad
iPhone
Watch
Vision
AirPods
TV & Home
Entertainment
Accessories
Support
0
+
The Apple Store Shopping Event
Get an Apple Gift Card with an eligible purchase.
1
Starts 11.28.
Learn more
iPhone 17 Pro
All out Pro.
Learn more
Buy
iPhone Air
The thinnest iPhone ever. With the power of pro inside.
Learn more
Buy
Give something special.
Find what they’ve been waiting for all year.
Shop gifts
Give something special.
Find what they’ve been waiting for all year.
Shop gifts
AirPods Pro 3
The world’s best in-ear Active Noise Cancellation.
Learn more
Buy
iPad Air
Now supercharged by the M3 chip.
Learn more
Buy
Apple Watch Series 11
The ultimate way to watch your health.
Learn more
Buy
MacBook Pro 14”
Supercharged by M5.
Learn more
Buy
Any condition carrier deals are here.
Select carriers accept eligible trade-in devices in any condition. Other offers available.
2
Find your deal
Apple Trade In
Get up to $180–$670 in credit when you trade in iPhone 1

In [28]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short neat serious brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [22]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += generate_page_data_with_links(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt






In [23]:
brochure_user_prompt("apple", "https://apple.com")


'\nYou are looking at a company called: apple\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## landing page\n\nApple\n\nApple\nApple\nStore\nMac\niPad\niPhone\nWatch\nVision\nAirPods\nTV & Home\nEntertainment\nAccessories\nSupport\n0\n+\nThe Apple Store Shopping Event\nGet an Apple Gift Card with an eligible purchase.\n1\nStarts 11.28.\nLearn more\niPhone 17 Pro\nAll out Pro.\nLearn more\nBuy\niPhone Air\nThe thinnest iPhone ever. With the power of pro inside.\nLearn more\nBuy\nGive something special.\nFind what they’ve been waiting for all year.\nShop gifts\nGive something special.\nFind what they’ve been waiting for all year.\nShop gifts\nAirPods Pro 3\nThe world’s best in-ear Active Noise Cancellation.\nLearn more\nBuy\niPad Air\nNow supercharged by the M3 chip.\nLearn more\nBuy\nApple Watch Series 11\nThe ultimate way to watch your health.\nLearn more\nBuy\nMa

In [29]:
def generate_brochure(company_name, url):
    user_prompt = brochure_user_prompt(company_name, url)
    response = client.chat.completions.create(
        model=Model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    display(Markdown(response.choices[0].message.content))



In [31]:
generate_brochure("apple", "https://apple.com")

# Apple: Innovating Technology, Enriching Lives

## About Us
Apple Inc. is a global leader in technology, known for its innovative products and services that enhance everyday life. With a commitment to excellence, Apple designs, develops, and sells consumer electronics, software, and online services. Our extensive product line includes iconic devices like the iPhone, iPad, Mac, Apple Watch, and AirPods, each designed with cutting-edge technology and user-friendly interface.

## Our Values
At Apple, our corporate culture prioritizes integrity, diversity, inclusion, and environmental stewardship. We believe in creating technology that is accessible to all, and we take serious measures to ensure our products are produced responsibly. Our initiatives focus on racial equity and justice, as well as protecting our planet, demonstrating our commitment to fostering a better world.

## Career Opportunities
We invite talented individuals to join our team and contribute to our mission. Apple offers robust career paths for various disciplines, from retail to technology and innovation. At Apple, you will not just be part of a company; you will be part of a community that values creativity, inspiration, and innovation. Explore our diverse opportunities and find your fit in a supportive environment that encourages personal growth and collective success.

## Our Customers
Apple’s products reach a diverse range of customers, including individuals, families, educators, and businesses across the globe. We are committed to understanding and meeting our customers' needs, fostering loyalty through superior customer service, support, and innovative solutions.

## Join Us
At Apple, you’ll have the chance to work on groundbreaking technology, inspire change, and make a difference. Our recruitment efforts focus on building a talented workforce that reflects the diverse world we serve. Join us for a rewarding career, where you can engage your curiosity, embrace challenges, and redefine what is possible.

Experience the excitement of working at Apple, where imagination meets innovation!

---

For more information about our products, values, and careers, visit our website or contact us directly.

In [35]:



def stream_brochure(company_name, url):
    stream = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)
    

In [36]:
stream_brochure("apple", "https://apple.com")

# Apple Inc.  
**Innovation, Excellence, and Impact**

---

## About Apple  
Apple Inc. is a global technology leader, designing, manufacturing, and marketing a diverse portfolio of innovative products and services that enrich lives. From the revolutionary iPhone and MacBook Pro powered by cutting-edge M-series chips, to the world’s best in-ear AirPods Pro, and the Apple Watch Series 11 enhancing health and wellness — Apple continues to redefine the limits of technology and user experience.

Apple offers seamless integration across its ecosystem including Mac computers, iPads, iPhones, smart watches, entertainment via Apple TV, and a broad range of accessories designed to elevate everyday life.

---

## Products & Services  
- **iPhone 17 Pro & iPhone Air:** The pinnacle of smartphone technology with pro-level power in the thinnest designs ever.  
- **MacBook Pro 14" with M5 chip:** Supercharged performance for professionals and creatives.  
- **iPad Air with M3 chip:** Powerful, portable, and versatile for work and play.  
- **Apple Watch Series 11:** Ultimate health tracking and connectivity on your wrist.  
- **AirPods Pro 3:** Leading Active Noise Cancellation and immersive sound.  
- **Apple TV & Entertainment:** Exclusive content and a rich media ecosystem.  

Apple also provides trade-in programs rewarding users for upgrading their devices responsibly, and carrier deals that offer benefits regardless of device condition.

---

## Company Culture  
Apple thrives on collaboration where **individual imagination meets shared values** to produce exceptional work. The company fosters an inclusive, diverse environment where every employee is encouraged to contribute authentically and innovate fearlessly. The culture is driven by a commitment to quality, sustainability, and making a positive impact globally.

### Key Values  
- Innovation and creativity  
- Diversity and Inclusion  
- Environmental responsibility  
- Excellence in customer experience

---

## Careers at Apple  
Apple invites talented individuals to **“Join a team and inspire the work.”** The company offers a variety of roles across many disciplines, locations, and teams including retail, engineering, design, marketing, and more. Employees benefit from comprehensive programs supporting life balance, diversity networks, personal growth, and career advancement.

### Why Work at Apple?  
- Be part of a global innovator driving technology forward  
- Contribute to products and services that impact millions worldwide  
- Inclusive and supportive environment championing individuality  
- Competitive benefits and development opportunities  

Discover your path and make an impact at Apple — where your imagination joins a collective force to create the extraordinary.

---

## For Customers & Partners  
Apple focuses relentlessly on quality and customer satisfaction, offering seamless shopping experiences both online and in retail locations globally. Regular promotional events like the Apple Store Shopping Event provide added value through gift cards and trade-in credits. Partner carriers and trade-in options make the ecosystem accessible and upgrade pathways rewarding.

Explore the future of technology with Apple — where every product reflects precision, performance, and passion.

---

**Contact & Explore More**  
Visit apple.com to shop latest products, learn about career opportunities, or explore Apple’s ecosystem and support services.

---

*Apple — Think Different. Experience the Future.*

In [39]:
def return_generate_brochure(company_name, url):
    user_prompt = brochure_user_prompt(company_name, url)
    response = client.chat.completions.create(
        model=Model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    return response.choices[0].message.content

In [44]:
def translate_brochure(language,company_name,url):
    user_prompt = f"""
Translate the following brochure into {language}:
{return_generate_brochure(company_name,url)}
"""
    response = client.chat.completions.create(
        model=Model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    display(Markdown(response.choices[0].message.content))





In [45]:

translate_brochure("Telugu", "apple", "https://apple.com")

# యాపిల్ కంపెనీ పత్రిక

## యాపిల్ గురించి

యాపిల్ ఇంక్. ఒక ప్రపంచవ్యాప్తంగా టెక్నాలజీలో ప్రముఖ నిర్మాణ సంస్థ, ఐఫోన్, ఐప్యాన్, మాక్, యాపిల్ వాచ్, మరియు ఎయిర్ పోడ్స్ వంటి ప్రాధమిక ఉత్పత్తుల ప్రবর্তనకు పేరు పొందింది. నాణ్యత మరియు వినియోగదారుడు కేంద్రిత డిజైన్‌కు ప్రాముఖ్యత నిస్తూ, యాపిల్ వ్యక్తిగత కంప్యూటింగ్ మరియు డిజిటల్ అనుభవాల భవిష్యత్తును ప్రాతిపాదిస్తూ ముందుకు సాగుతుంది.

## మన విలువలు

యాపిల్‌లో, మన మూలక విలువలు అందుబాటులో ఉండటం, గోప్యత, చేర్చడం, విభిన్నత మరియు పర్యావరణ స్థిరత్వం పరిసరాలు చుట్టూ తిరుగుతాయి. ప్రతి ఒక్కరికీ అవసరమైన ఉత్పత్తులను సృష్టించడంలో మేము విశ్వాసం కలిగి ఉన్నాము, మన సాంకేతికతలు అందరూ అందుబాటులో మరియు ఉపయోగించగలుగుతున్నట్లు నిర్ధారిస్తూ.

## కస్టమర్లు

యాపిల్ విభిన్న కస్టమర్లను అందిస్తుంది, వ్యక్తిగత వినియోగదారులు నుండి వ్యాపారాలు మరియు విద్యా సంస్థలకు. కట్టుతారు, ఈచిన. ప్రోత్సాహం, నూతన సాంకేతికత ద్వారా ప్రజల జీవితాలను మెరుగుపరచడం.

## కంపెనీ సంస్కృతి

యాపిల్‌లో, ఆమోదం, సహోద్యోగం మరియు వ్యక్తిత్వానికి గౌరవం ప్రధాన లక్షణాలు. వివిధతను స్ఫూర్తి పరచడం మరియు స్రవంతి ప్రోత్సహించడం ద్వారా, మన టీం సభ్యులు తమ ఆలోచనలను ముక్కున సృష్ఠించనకి విద్వేషించారు. యాపిల్‌లో, మీరు మరొక ఉద్యోగి కావడంలేదు; మీరు మార్పును నడిపించే మరియు ప్రపంచాన్ని ప్రేరేపించే దృష్టాశీల టీం భాగం.

## యాపిల్‌లో కెరీర్స్

యాపిల్ వివిధ రంగాలలో, కొన్నిసార్లు కస్టమర్ సేవ, ఇంజనీరింగ్, డిజైన్ మరియు మరిన్ని వంటి విస్తృత కెరీర్ అవకాశాలను అందించుడు. కొత్త ఉద్యోగులను మేము మా అధికారులు, ఆలోచనలు మరియు పరికరాల సముదాయంలోకి దెరకులు తెరవడం కాదవగా అందించిట్ట్. ఇక్కడ, మీ వ్యక్తిగత మరియు వృత్తిపరమైన అభివృద్ధికి అవసరమైన మద్దతు మరియు వనరులను సంపాదిస్తారు.

మానవతను వెనక్కి నైపుణ్యం ఆశిస్తాను, ఇక్కడ మనకు ఉత్పత్తి అద్భుతమైన కంటెంట్ మీకు ఉంది. మేము ఉన్నారు, పరిచయం ఉంచండి క్రొత్త ఉద్యోగాలు ఆమోదించాలని వెళ్ళండి స్పష్టంగా అందించడం, యాపిల్ జీవనాన్ని ఎలా పొందె హోదాయని చూసి పొందితారు.

## మమ్మల్ని సంప్రదించండి

యాపిల్ గురించి మరింత తెలుసుకోవాలనుకుంటున్నారా? దయచేసి మా వెబ్‌సైట్‌ని సందర్శించండి లేదా నేరుగా మమ్మల్ని సంప్రదించండి. మేము ఎప్పుడూ కస్టమర్లతో, పెట్టుబడిదారులతో మరియు సంభావ్య టీం సభ్యులతో కనెక్ట్ అవ్వడానికి మార్గాలను వెతుకుతున్నాము.

భవిష్యత్తును అన్వేషించండి. యాపిల్‌ని అన్వేషించండి.

In [48]:
def translate_brochure_stream(language,company_name,url):
    user_prompt = f"""
Translate the following brochure into {language}:
{return_generate_brochure(company_name,url)}
"""
    stream = client.chat.completions.create(
        model=Model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True
    )
    response=""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)



In [49]:
translate_brochure_stream("Telugu", "apple", "https://apple.com")

# ఆపిల్ కంపెనీ బ్రోచర్

## ఆపిల్ గురించి
ఆపిల్ ఇన్‌క్ ఒక గ్లోబల్ టెక్నాలజీ నేతృత్వ సంస్థ, ఇది దాని నూతనాత్సవమయ మరియు ఆధునిక ఉత్పత్తుల కోసం ప్రసిద్ధి చెందింది. ఐఫోన్, మాక్, ఐప్యాడ్, ఆపిల్ వాచ్ మరియు ఎయిర్‌పాడ్స్ వంటి అధిక నాణ్యత ఉత్పత్తులను అందించే విధానంలో మనం నిబ్బరంగా ఉన్నాం. డిజైన్ మరియు చర్యలపై నిరంతర దృష్టితో, ఆపిల్ టెక్ పరిశ్రమలో మన్నదని స్థాయి స్థాపించడానికి ముందు కదులుతున్నది.

## ఉత్పత్తులు & సేవలు
- **ఐఫోన్**: శక్తిమంతమైన ఐఫోన్ 17 ప్రొ మరియు అతి పుల్లైన ఐఫోన్ ఎయిర్ వంటి తాజా మోడళ్లను కనుగొనండి.
- **మాక్**: ఆధునిక M5 టెక్నాలజీ ద్వారా శక్తిమంతమైన కొత్త మ్యాక్బుక్ ప్రోతో అద్భుత పనితీరు అనుభవించండి.
- **ఐప్యాడ్**: M3 చిప్‌తో సూపర్‌చార్జ్ చేయబడిన ఐప్యాడ్ ఎయిర్ శక్తి మరియు పోర్టబులిటీని కలగలుపుతుంది.
- **ఆపిల్ వాచ్**: శాశ్వత వైద్య పరిశీలన పరికరం, వినూత్న లక్షణాలతో కూడి ఉంది.
- **ఎయిర్‌పాడ్స్**: ప్రపంచంలోనే అత్యుత్తమ ఇన్-ఇయర్ యాక్టివ్ నాయిస్ క్యాన్సలేషన్‌తో ప్రీమియమ్ శ్రవణ నాణ్యతను ఆస్వాదించండి.

## కంపెనీ సంస్కృతి
ఆపిల్‌లో, మేము సృజనాత్మకత మరియు నవోత్పత్తి సంస్కృతిని養护 చేస్తాము. వివిధత విజయాన్ని నడిపిస్తుందని మేము నమ్ముతాము. మా సహకార వాతావరణం ఆకర్షితులను స్వేచ్చగా తమ ఆలోచనలను వ్యక్తం చేయడానికి మరియు వారి ప్రత్యేకతను స్వీకరించడానికి ప్రోత్సహిస్తుంది. మా సిబ్బందిలో సమానత్వం, సమానత మరియు పర్యావరణం వంటి విలువలను ప్రాతినిథ్యం వహిస్తూ, మా కార్యాలయం మేము నివసిస్తున్న విభిన్న ప్రపంచాన్ని ప్రతిబింబిస్తుంది.

## ఆపిల్‌లో కెరీర్‌లు
ఆపిల్‌లో చేరండి, ఇక్కడ మీరు టెక్నాలజీ భవిష్యత్తును ప్రేరేపించే మరియు ఆకృతీకరించే టీమ్‌లో భాగమవుతారు. మేము వివిధ టీమ్స్ మరియు ప్రదేశాలలో విస్తృత కిర్యలను అందిస్తున్నాము, టాలెంట్ పుష్కలంగా ఉండే వాతావరణాన్ని ప్రోత్సహిస్తున్నాము. మాకు చేరండి మరియు మీ ఉనికికి అర్ధం ఉండేది—మీరు ఉండండి, మరియు మా అత్యుత్తమ ఉత్పత్తులను రూపొందించడానికి మీ ప్రత్యేక దృష్టిని తెచ్చుకోండి.

### అవకాశాలు కలిగి ఉంటాయి:
- ఆపిల్ రీటైల్
- ఇంజనీరింగ్
- డిజైన్
- మార్కెటింగ్
- నిర్వహణ

## కస్టమర్ కట్టవలసిన పరామర్శ
మా కస్టమర్లు మేము చేసిన ప్రతి వేళకు గుండెల్లో ఉన్నారు. అందువల్ల, మేము కేవలం ఉత్పత్తులు ఇవ్వడం మాత్రమే కాకుండా, భవిష్యత్తు సమాజాన్ని అనుసంధానం చేసేటట్లు అద్భుతమైన అనుభవాలను అందించే అవకాశం కలిగి ఉన్నం. మాకు నిష్ఠాత్మక మద్దతు వ్యవస్థ మరియు వనరులు అందుబాటులో ఉన్నందున, ప్రతి ఆపిల్ అనుభవం మాకు ఉన్న ఉన్నతమైన సేవా ప్రమాణాలను గుర్తించడానికి చేస్తాము.

## మాతో పెట్టుబడి వేస్తే
ఒక పబ్లిక్ పర్యవేక్షిత కంపెనీగా, ఆపిల్ నిత్యం అభివృద్ధి మరియు నవోత్పత్తి కోసం ప్రయత్నిస్తోంది, ఇది సుస్థిర మరియు వ్యూహాత్మక అభివృద్ధుల కోసం పెట్టుబంచేసేవారికి ఆకర్షణీయమైన అవకాశంగా మారుతోంది.

## మాతో సంప్రదించండి
మరింత సమాచారం కోసం, మా వెబ్‌సైట్‌ను అన్వేషించండి లేదా మా మద్దతు ఛానల్స్ ద్వారా చేరుకోండి. టెక్నాలజీ భవిష్యత్తును ఆకృతీకరించడానికి మాతో చేరండి—కొందరు కలిసి మలో మంచి ప్రేరణను తీసుకురాగలుగుతాం.

In [50]:
stream_brochure("formula1", "https://f1.com")

# Formula 1 (F1) Brochure

---

## Overview

**Formula 1® (F1)** is the world’s premier international motorsport racing series, renowned for its high-speed excitement, cutting-edge technology, and thrilling competition. F1 holds globally watched Grand Prix events, culminating in the crowning of the World Drivers' and Constructors' Champions each year. The 2025 season is packed with exhilarating races across iconic locations like Brazil, Las Vegas, Qatar, and Abu Dhabi.

---

## What We Do

- Host a global schedule of 24+ Grand Prix events annually  
- Showcase the pinnacle of automotive engineering and driver skill  
- Deliver an immersive fan experience through F1 TV, fantasy leagues, esports, and official merchandise  
- Provide premium hospitality and unique fan experiences at races worldwide  

---

## Our Customers & Fans

- Motorsports enthusiasts, ranging from casual followers to devoted fans  
- Gaming and esports communities engaged in F1 digital experiences (F1 Manager, F1 Clash, F1 Mobile Racing)  
- Corporate clients and VIPs seeking premium race hospitality and experiences  
- Collectors and fans purchasing official authentic merchandise through the F1 Store  

---

## Company Culture & Values

Though F1’s primary focus is on thrilling racing and entertainment, it embraces:

- **Innovation:** Constantly pushing the limits of automotive technology and race broadcasting.  
- **Diversity:** Featuring a roster of international drivers from varied backgrounds competing at the highest level.  
- **Engagement:** Building a global community through fan interaction, fantasy leagues, and esports.  
- **Excellence:** Upholding rigorous standards for safety, fair competition, and sporting integrity.  

---

## Careers at F1

F1 offers diverse career opportunities in areas including:

- Motorsport engineering and team operations  
- Digital media production and broadcast technology  
- Marketing, sponsorship, and event management  
- Hospitality and fan engagement experiences  
- Software development and gaming/esports management  

If you are passionate about motorsport, innovation, and global entertainment, F1 provides a platform to grow your skills in an exciting international environment. Check the official F1 website for updated job listings and internship opportunities.

---

## Leadership & Governance

F1 operates under the aegis of the **FIA** (Fédération Internationale de l'Automobile) ensuring strict compliance with sporting codes and safety regulations. The leadership team of Formula 1 is focused on expanding the sport’s global reach, enhancing fan experiences, and promoting sustainability within motorsport.

---

## Connect With Formula 1

- **Website:** [formula1.com](https://www.formula1.com)  
- **F1 TV:** Stream every race live with exclusive content  
- **Fantasy & Gaming:** Join leagues and esports competitions  
- **Store:** Official merchandise and collectibles  
- **Social Media:** Engage with us for race updates, driver interviews, and behind-the-scenes content  

---

Experience the speed, precision, and passion of Formula 1 — where legends are made and the future of racing unfolds.

---

*Official Home of Formula 1® Racing*  
© Formula One Licensing B.V. 2024

In [54]:
def genImages_s_with_brochure(company_name,url):
    user_prompt = f"""
"add text diagrams & make it beautiful to the brochure provided"
{return_generate_brochure(company_name,url)}
"""
    response = client.chat.completions.create(
        model=Model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    display(Markdown(response.choices[0].message.content))

In [55]:
genImages_s_with_brochure("formula1", "https://f1.com")

# Formula 1® - The Pinnacle of Motorsport

## About Us
Welcome to Formula 1®, where the world of motorsport transcends into high-speed excitement and cutting-edge technology. We are dedicated to delivering thrilling experiences for both fans and participants. With races held at iconic global circuits, we have fostered a culture of innovation, competition, and teamwork.

---

### **Company Culture**
At Formula 1, we thrive on **passion** and **dedication**. 

- **Collaboration:** Our team pushes boundaries to redefine excellence in racing.
- **Diversity & Inclusivity:** We value diverse perspectives, fostering an environment ripe for creativity and open idea sharing.
- **Community Focus:** Together, we aim to create unforgettable moments for fans and communities worldwide.

![](https://via.placeholder.com/400x200?text=Teamwork+and+Innovation)

---

### **Our Customers**
Formula 1 caters to a **global audience** consisting of:

- **Die-Hard Motorsport Enthusiasts:** Committed fans who pulse with every twist and turn on the track.
- **Casual Viewers:** Individuals who tune in for the excitement and spectacle of racing.
  
Our fans are central to everything we do, and we cultivate connections through:

- **Exciting Race Experiences:** Engaging live events and immersive hospitality.
- **F1 TV Platform:** Digital access to racing from anywhere in the world.
- **Comprehensive Race Schedules:** Keeping fans up-to-date with exclusive events and content.

![](https://via.placeholder.com/400x200?text=Engaging+with+Fans)

---

### **Careers at Formula 1**
**Join a team where innovation meets passion!**

We are continuously seeking talented individuals across various fields, including:

- **Engineering**
- **Marketing**
- **Event Management**

At Formula 1, you will discover:

- **Professional Growth:** Opportunities to enhance your skills.
- **Industry Leaders:** Work alongside some of the best in the industry.
- **Excitement of a Global Sport:** Be part of an internationally recognized legacy.

If you’re ready to accelerate your career, visit our **careers page** to explore current job openings and embark on your journey with us!

![](https://via.placeholder.com/400x200?text=Join+Us)

---

### **Experience Formula 1**
From **thrilling race weekends** to **interactive fan experiences**, Formula 1 offers unique opportunities for everyone.

- **Live Race Tickets:** Experience the action firsthand.
- **Hospitality Options:** Enjoy an unforgettable race day.
- **F1 TV:** Stay updated with the latest news, results, and standings through our digital platform.

Join us at Formula 1, where **passion meets precision**, and the thrill of racing comes to life! 

--- 

For more information and to connect with us, visit our official website: [Formula 1®](https://www.formula1.com)